# Data transformation

Transform data in json

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium

%matplotlib inline

## Per country

In [3]:
BASE_FOLDER_TEMP = '../data/climate-change-earth-surface-temparture-data/'

In [3]:
FILENAME = 'GlobalLandTemperaturesByCountry.csv'
df = pd.read_csv(BASE_FOLDER_TEMP + FILENAME)

In [4]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


In [5]:
df.Country.value_counts().count()

243

In [10]:

tmp = df[df.Country == 'Antarctica'].AverageTemperature
tmp.isnull().sum() / 

0.0

In [42]:
date = '1800-01-01'
df[df.dt == date].count()

dt                               76
AverageTemperature               65
AverageTemperatureUncertainty    65
Country                          76
dtype: int64

In [41]:
date = '1850-01-01'
df[df.dt == date].count()

dt                               192
AverageTemperature               185
AverageTemperatureUncertainty    185
Country                          192
dtype: int64

In [48]:
date = '1850-01-01'
tmp = df[df.dt == date].AverageTemperature
tmp.isnull().sum() / tmp.count()

0.03783783783783784

In [137]:
date = '1850-01-01'
remain = df[df.dt >= date].copy().reset_index()
remain.set_index(['Country'], inplace=True)
remain.drop(['AverageTemperatureUncertainty','index'], axis=1, inplace=True)
r = remain.groupby('Country'). \
    apply(lambda x: (x.dt[x.dt.astype(str).str.contains("^.*-01-01$")] + "_" + x.AverageTemperature[x.dt.astype(str).str.contains("^.*-01-01$")].astype(str)).tolist())

index                            465370
dt                               465370
AverageTemperature               452458
AverageTemperatureUncertainty    453197
Country                          465370
dtype: int64


In [140]:
import json

OUTNAME = 'country_temp_from_%s_step_1y.json' % date
OUTPATH = '../app/data/'
with open(OUTPATH + OUTNAME, 'w') as f:
    json.dump(r.to_dict(), f, ensure_ascii=False)

## Global

In [4]:
FILENAME = 'GlobalTemperatures.csv'
df = pd.read_csv(BASE_FOLDER_TEMP + FILENAME)

In [5]:
df.head()

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,8.490,2.451,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,11.573,2.072,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
date = '1850-01-01'
remain = df[df.dt >= date].copy().reset_index()
remain = remain[remain.dt.astype(str).str.contains("^.*-01-01$")].reset_index()[['dt','LandAverageTemperature']].copy()
remain['World'] = remain['dt'].astype(str) + "_" + remain['LandAverageTemperature'].astype(str)
r = remain.drop(['dt', 'LandAverageTemperature'], axis=1)

In [42]:
import json

OUTNAME = 'glob_temp_from_%s_step_1y.json' % date
OUTPATH = '../app/data/'
with open(OUTPATH + OUTNAME, 'w') as f:
    json.dump(r.to_dict(orient='list'), f, ensure_ascii=False)